In [242]:
import os
import pandas as pd
from os import chdir
print(os.getcwd())
chdir('C:\\Users\\eric\\Desktop')

C:\Users\eric\Desktop


In [243]:
doc=pd.read_csv(r"C:\Users\eric\Desktop\acquisitions-2020.csv",header=0,encoding='cp1252')

In [244]:
doc.head()

,Transaction Name,Acquiree Name,Acquirer Name,Acquiree Description,Acquirer's Description,Price,Announced Date,Acquirer_Categories,Acquiree_Categories
0,SABMiller PLC acquired by Anheuser-Busch InBev,SABMiller PLC,Anheuser-Busch InBev,SABMiller is a brewers with more than 200 beer...,Anheuser-Busch InBev is a brewer and provider ...,1.030000e+11,2015-10-12,"Brewing, Consumer Goods, Food and Beverage","Craft Beer, Hospitality"
1,Shire acquired by Takeda Pharmaceutical,Shire,Takeda Pharmaceutical,Shire biotechnology company that focuses on se...,Takeda Pharmaceutical Company Limited operates...,6.200000e+10,2018-04-25,"Biotechnology, Health Care, Health Diagnostics...","Biopharma, Biotechnology, Genetics, Health Car..."
2,Time Warner acquired by AT&T,Time Warner,AT&T,Time Warner is a media and entertainment compa...,AT&T is a telecommunications company that prov...,8.540000e+10,2016-10-22,"Mobile, Service Industry, Telecommunications, ...","Broadcasting, Content Delivery Network, Film"
3,Celgene acquired by Bristol-Myers Squibb,Celgene,Bristol-Myers Squibb,"CelgeneÂ discovers, develops and commercialize...","Bristol-Myers Squibb engages in the discovery,...",7.400000e+10,2019-01-03,"Biotechnology, Health Care, Medical, Pharmaceu...","Biotechnology, Medical"
4,21st Century Fox acquired by The Walt Disney C...,21st Century Fox,The Walt Disney Company,21st Century Fox is an American mass media cor...,The mission of The Walt Disney Company is to b...,7.130000e+10,2017-12-14,Animation,"Film, Market Research, Media and Entertainment..."


In [245]:
len(doc)

1000

In [246]:
doc=doc.dropna()

In [247]:
len(doc)

988

In [248]:
doc = doc.fillna('').astype(str).apply(lambda x: x.str.lower()) 

In [249]:
#import package
import re
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora,models
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from stop_words import get_stop_words

In [250]:
Lem = WordNetLemmatizer()
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(Lem.lemmatize(token,pos="n"))
    return result

In [251]:
processed_docs = doc['Acquiree Description'].map(preprocess)
processed_docs[:10]

0    [sabmiller, brewer, beer, brand, employee, cou...
1    [shire, biotechnology, company, focus, serving...
2    [time, warner, medium, entertainment, company,...
3    [celgeneâ, discovers, develops, commercializes...
4    [century, american, mass, medium, corporation,...
5          [allergan, global, pharmaceutical, company]
6         [sabic, diversified, manufacturing, company]
7    [aetna, diversified, health, care, benefit, co...
8    [manufacture, information, management, storage...
9    [express, script, operates, pharmacy, benefit,...
Name: Acquiree Description, dtype: object

In [252]:
type(processed_docs)

pandas.core.series.Series

In [253]:
data=processed_docs.to_frame(name='ipc')

In [254]:
data["filled_date"] = doc["Announced Date"]

In [255]:
data.head()

,ipc,filled_date
0,"[sabmiller, brewer, beer, brand, employee, cou...",2015-10-12
1,"[shire, biotechnology, company, focus, serving...",2018-04-25
2,"[time, warner, medium, entertainment, company,...",2016-10-22
3,"[celgeneâ, discovers, develops, commercializes...",2019-01-03
4,"[century, american, mass, medium, corporation,...",2017-12-14


In [256]:
data.to_csv("data.csv",index=False)

In [259]:
data=pd.read_csv(r"C:\Users\eric\Desktop\data.csv",header=0,encoding='cp1252')

In [260]:
data.head()

,ipc,filled_date
0,"sabmiller, brewer, beer, brand, employee, country",2015
1,"shire, biotechnology, company, focus, serving,...",2018
2,"time, warner, medium, entertainment, company, ...",2016
3,"celgeneÃ¢, discovers, develops, commercializes...",2019
4,"century, american, mass, medium, corporation, ...",2017


In [261]:
import csv
import time
import itertools
import numpy as np
# node edge
def MakingEdgeNodelist(keyword_vec,sep_sign,year):
    # (1) Keyword list (data set for combination)
    keywords_list =[]
    for words in keyword_vec:
        #print("words :", words)
        split_words = words.split(sep_sign) # sep_sign으로 구분된 데이터를 분리하여
        split_words = [ x.replace(" ","") for x in split_words]                                                # 리스트로 저장
        soted_words = sorted(split_words) # ordering
        keywords_list.append(soted_words)

    # (2) Combination
    comb_list = []
    for i in keywords_list:
        for subset in itertools.combinations(i, 2):
            if subset[0] != subset[1]: # 같은 단어의 컴비네이션 제외
                comb_list.append(subset)

    # (3) Make dict & count
    comb_count = {}
    for combi in comb_list:
        comb_count[combi] = comb_count.get(combi,0)+1

    # (4) dic to df
    comb_df = pd.DataFrame()

    ## (4-1) dic_key to list
    comb_keylist = []
    for key in comb_count.keys():
        combkey_tolist = list(key)
        comb_keylist.append(combkey_tolist)

    ## (4-2) dic_key split
    source_list = []
    target_list = []
    for i in range(len(comb_keylist)):
        source = comb_keylist[i][0]
        target = comb_keylist[i][1]

        source_list.append(source)
        target_list.append(target)

    comb_df['Source_Label'] = source_list
    comb_df['Target_Label'] = target_list
    comb_df['weight'] = comb_count.values()
    comb_df['year'] = year

    comb_df = comb_df.sort_values('Source_Label',axis=0).reset_index(drop=True)

    # (5) node data
    total_tech_list = []
    for tech_li in keywords_list:
        tech = list(set(tech_li))
        total_tech_list.extend(tech)
    # node_data = pd.concat([comb_df['Source_Label'],comb_df['Target_Label']],axis=0) # 총 출현 기술 ; 잘못됨을 확인함

    node_df = pd.DataFrame()

    node_count = {}
    for node in total_tech_list:
        node_count[node] = node_count.get(node,0)+1

    node_df['label'] = node_count.keys()
    node_df['count'] = node_count.values()
    node_df['id'] = node_df.index
    node_df['year'] = year

    # (6) make id
    node_id = node_df[['label','id']]
    node_id.columns = ['Source_Label','Source']

    merge_sid = pd.merge(comb_df,node_id,how = 'inner')

    node_id.columns = ['Target_Label','Target']

    merge_tid = pd.merge(merge_sid,node_id,how = 'inner') # merge 과정에서 순서가 바뀜

    edge_df = merge_tid.sort_values(['Source_Label','Target_Label'])

    edge_df['Type'] = ['Undirected']*len(edge_df)

    #edge_cols = ['Source','Target','weight','Type','Source_Label','Target_Label']
    #edgelist = edgelist[edge_cols]

    return(edge_df,node_df)


# ipc 개수 2 이상인 데이터 필터링
ipc_len = [ len(str(x).split(",")) for x in data.ipc]
data['ipc_len'] = ipc_len

dataset = data.loc[data.ipc_len>1]

# edge / node list
edge_list = pd.DataFrame()
node_list = pd.DataFrame()

s_time = time.time()
for year in range(2007,2019):
    doc = dataset.loc[dataset['filled_date']== year,'ipc']
    
    edge_part,node_part = MakingEdgeNodelist(doc,",",year)
    
    edge_list = pd.concat([edge_list,edge_part],axis=0)
    
    node_list = pd.concat([node_list,node_part],axis=0)
    
e_time = time.time()
print("Execution_time :",e_time - s_time)

#

print(node_list.head())
print(edge_list.head())

Execution_time : 0.24501895904541016
      label  count  id  year
0    device    1.0   0  2010
1     serve    1.0   1  2010
2  develops    1.0   2  2010
3      need    1.0   3  2010
4      care    5.0   4  2010
    Source Source_Label  Target   Target_Label        Type  weight  year
73     151     absolute     149          asset  Undirected     1.0  2010
71     151     absolute     145          class  Undirected     1.0  2010
5      151     absolute     152  discretionary  Undirected     1.0  2010
0      151     absolute     150     investment  Undirected     1.0  2010
66     151     absolute     146           long  Undirected     1.0  2010


C:\Users\eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:108: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [262]:
node_list["count"] = node_list["count"] .astype(int)
edge_list["weight"] = edge_list["weight"] .astype(int)

In [265]:
edge_list.sort_values(by='weight',ascending=False).head(50)

,Source,Source_Label,Target,Target_Label,Type,weight,year
377,75,provides,18,service,Undirected,15,2015
213,162,provides,164,service,Undirected,13,2018
376,31,provider,18,service,Undirected,12,2015
1340,14,provides,75,service,Undirected,11,2016
450,137,provider,13,service,Undirected,11,2017
451,10,provides,13,service,Undirected,11,2017
356,16,company,13,service,Undirected,9,2017
1324,142,estate,143,trust,Undirected,8,2018
1933,3,company,115,focused,Undirected,8,2018
1338,145,real,143,trust,Undirected,8,2018


In [264]:
node_list=node_list.drop_duplicates()
edge_list=edge_list.drop_duplicates()

In [227]:


# year form trans
#node_data['time'] = [ str(x)  for x in node_list.year]
#edge_data['time'] = [ str(x)  for x in edge_list.year]
#print(node_data.head())
# =========== tiemstamp form ================= #

# [1] node

# ===== 1. time stamp format 변환 ===== #
node_dict = {} 
node_dict2 = {}
n_year_list = []
n_label_list = list(set(node_list['label']))

# time_stamp
for i in n_label_list:
    ny = node_list[node_list['label']==i]['year']
    node_dict[i] = list(ny)
# time_score_stamp
n_time_score = []
for i in range(len(node_list)):
    ny = node_list['year'].iloc[i]
    nc = node_list['count'].iloc[i]
    nt = str(ny) + "&" + str(nc)
    n_time_score.append(nt)
node_list['time_score'] = n_time_score

for i in n_label_list:
    nt = node_list[node_list['label']==i]['time_score']
    re_nt = [ "[" + x + "]" for x in nt]
    node_dict2[i] = list(re_nt)

# ===== 2. 변환한 속성 merge ==== #
n_df = pd.DataFrame()
n_df['label'] = node_dict.keys()
n_df['time_stamp'] = node_dict.values()
n_df['time_score_stamp'] = node_dict2.values()
#print(n_df.head())
#print(n_df['time_stamp'][0])

# ===== 3. 타입변환 및 replace ==== #
# 1) time stamp
n_df['time_stamp'] = [ "<" + str(x) + ">" for x in n_df['time_stamp']]
n_df['time_stamp'] = [ x.replace("'","") for x in n_df['time_stamp']]
print(n_df['time_stamp'][0])

# 1) time socre stamp
n_df['time_score_stamp'] = [ str(x)[1:-2] for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ "<" + x + ">" for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace("'","") for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace(",",";") for x in n_df['time_score_stamp']]
n_df['time_score_stamp'] = [ x.replace("&",",") for x in n_df['time_score_stamp']]

print(n_df['time_score_stamp'][0])

# 2) string replace



<[2015]>
<[2015,1]>


In [228]:
    
# [2] edge
# ===== 0. name column 생성 ===== #
edge_list['t_label'] = edge_list['Source_Label'] + "&" + edge_list['Target_Label']

# ===== 1. time stamp format 변환 ===== #
edge_dict = {}
edge_dict2 = {}
e_year_list = []
e_label_list = list(set(edge_list['t_label']))

# time_stamp
for i in e_label_list:
        ey = edge_list[edge_list['t_label']==i]['year']
        edge_dict[i] = list(ey)


In [229]:
    
# time_score_stamp
e_time_score = []
for i in range(len(edge_list)):
        ey = edge_list['year'].iloc[i]
        ec = edge_list['weight'].iloc[i]
        et = str(ey) + "&" + str(ec)
        e_time_score.append(et)
edge_list['time_score'] = e_time_score

for i in e_label_list:
        et = edge_list[edge_list['t_label']==i]['time_score']
        re_et = [ "[" + x + "]" for x in et]
        edge_dict2[i] = list(re_et)

# ===== 2. 변환한 속성 merge ==== #
e_df = pd.DataFrame()
e_df['t_label'] = edge_dict.keys()
e_df['time_stamp'] = edge_dict.values()
e_df['time_score_stamp'] = edge_dict2.values()

# ===== 3. 타입변환 및 replace ==== #
# 1) time stamp
e_df['time_stamp'] = [ "<" + str(x) + ">" for x in e_df['time_stamp']]
e_df['time_stamp'] = [ x.replace("'","") for x in e_df['time_stamp']]
#print(n_df['time_stamp'][0])

# 1) time socre stamp
e_df['time_score_stamp'] = [ str(x)[1:-2] for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ "<" + x + ">" for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace("'","") for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace(",",";") for x in e_df['time_score_stamp']]
e_df['time_score_stamp'] = [ x.replace("&",",") for x in e_df['time_score_stamp']]

#print(e_df['time_score_stamp'][0])

# [3] index
n_df = n_df.sort_values(by='label')
n_df['id'] = range(len(n_df))

e_df['label'] = [str(x).split("&")[0] for x in e_df['t_label']]
e_df = pd.merge(e_df,n_df[['id','label']],how='left',on='label')
e_df.columns = ['t_label','time_stamp','time_score_stamp','Source_label','Source']

e_df['label'] = [str(x).split("&")[1] for x in e_df['t_label']]
e_df = pd.merge(e_df,n_df[['id','label']],how='left',on='label')
e_df.columns = ['t_label', 'time_stamp', 'time_score_stamp', 'Source_label', 'Source',
       'Target_label', 'target']

e_df = e_df.drop(['t_label'],axis=1)



In [237]:
e_df.rename(columns={'time_score_stamp':'weight'},inplace=True)

In [238]:
e_df.head()

,time_stamp,weight,Source_label,Source,Target_label,target
0,<[2017]>,"<[2017,1]>",luxembourg,1354,product,1769
1,<[2015]>,"<[2015,1]>",colorado,469,nebraska,1518
2,<[2014]>,"<[2014,1]>",real,1841,retail,1900
3,<[2016]>,"<[2016,1]>",agriculture,62,growing,1029
4,<[2017]>,"<[2017,1]>",nationwide,1512,pub,1795


In [232]:
# save file
n_df.to_csv("des_crunch_node.csv",index=False)
e_df.to_csv("des_crunch_edge.csv",index=False)

In [239]:
e_df.head()

,time_stamp,weight,Source_label,Source,Target_label,target
0,<[2017]>,"<[2017,1]>",luxembourg,1354,product,1769
1,<[2015]>,"<[2015,1]>",colorado,469,nebraska,1518
2,<[2014]>,"<[2014,1]>",real,1841,retail,1900
3,<[2016]>,"<[2016,1]>",agriculture,62,growing,1029
4,<[2017]>,"<[2017,1]>",nationwide,1512,pub,1795


In [241]:
e_df.tail(50)

,time_stamp,weight,Source_label,Source,Target_label,target
27164,"<[2014, 2017]>","<[2014,1]; [2017,1]>",amec,102,consultancy,522
27165,<[2014]>,"<[2014,2]>",manufacturing,1383,specialty,2095
27166,<[2018]>,"<[2018,1]>",antelliq,128,produce,1766
27167,<[2014]>,"<[2014,1]>",business,330,ethically,822
27168,<[2011]>,"<[2011,1]>",gold,1009,metal,1436
27169,<[2015]>,"<[2015,1]>",processing,1764,sungard,2159
27170,<[2013]>,"<[2013,1]>",developing,660,specialty,2095
27171,<[2018]>,"<[2018,1]>",robotics,1924,safer,1944
27172,<[2016]>,"<[2016,1]>",mining,1459,produce,1766
27173,<[2016]>,"<[2016,1]>",built,328,company,489
